In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import cv2
from pathlib import Path
from skimage.io import imread, imsave
from skimage.transform import resize

import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras import layers as L
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import RMSprop, Adam
from keras.applications import vgg16
from keras.layers import Flatten, Conv2D

In [2]:
training_data = '/Users/pawankumarkc/Downloads/Datasets_to_delete/tomato_leaf_images/train'
validation_data = '/Users/pawankumarkc/Downloads/Datasets_to_delete/tomato_leaf_images/val'

In [3]:
#Lebel the files name by using integer

labels_dict = {'AmericanLeafMiner':0, 
               'Healthy':1, 
               'MagnesiumDeficiency':2, 
               'SerpentineLeafMiner':3 }



In [4]:
type([f for f in os.listdir(training_data) if f!='.DS_Store'])

list

In [5]:
type(os.listdir(training_data))

list

In [9]:
#Creating dataframe for training dataset

train_df = []

for folder in os.listdir(training_data):
    imgs_path = training_data / folder
    #Get the list of all images stored in that directory
    imgs = sorted(imgs_path.glob('*.jpg'))

    #Store each image path and corresponding label
    for img_name in imgs:
        train_df.append((str(img_name), labels_dict[folder]))

train_df = pd.DataFrame(train_df, columns=['image', 'label'], index=None)

#Shuffel the dataset
train_df = train_df.sample(frac=1.).reset_index(drop=True)

KeyboardInterrupt: 

In [ ]:
#Creating dataframe for test dataset

valid_df = []

for folder in os.listdir(validation_data):
    imgs_path = validation_data / folder
    #Get the list of all images stored in that directory
    imgs = sorted(imgs_path.glob('*.jpg'))

    #Store each image path and corresponding label
    for img_name in imgs:
        valid_df.append((str(img_name), labels_dict[folder]))

valid_df = pd.DataFrame(valid_df, columns=['image', 'label'], index=None)

#Shuffel the dataset
valid_df = valid_df.sample(frac=1.).reset_index(drop=True)

In [ ]:
#Configuration

#dimentions to consider for the image
img_rows, img_cols, img_channels = 224, 244, 3

#Batch size
batch_size = 8

#Total no. of classes
nb_classes = 4


In [ ]:
#Data augumentation (Alternative for DataImageGenerator)

import imgaug as ia
from imgaug import augmenters as iaa

seed = 1234
ia.seed(seed)

#Augumentation sqeuence 
seq = iaa.OneOf([
    iaa.Fliplr(), #Horizontal flip
    iaa.Affine(rotate=20), #Affine algo for rotation
    iaa.Multiply((1.2, 1.5)), #Zooming

])


In [ ]:
#Data generator

def data_generator(data, batch_size, is_validation_data = False, preprocessing_function = None):
    n = len(data)
    nb_batches = int(np.ceil(n/batch_size)) #ceil value gives above value(rounded of) of a given number
    indices = np.arange(n)

    while True:
        if not is_validation_data:
            np.random.shuffle(indices)

        for i in range(nb_batches):
            next_batch_indices = indices[i*batch_size:(i+1)*batch_size]
            nb_examples = len(next_batch_indices)

            #Define 2 numpy array for containing batch data and labels
            batch_data = np.zeros((nb_examples, img_rows, img_cols, img_channels), dtype=np.float32) 
            batch_labels = np.zeros((nb_examples, nb_classes), dtype=np.float32)

            #Process the next batch 
            for j, idx in enumerate(next_batch_indices):
                img = cv2.imread(data.iloc[idx]['image'])
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                label = data.iloc[idx]['label']

                if not is_validation_data:
                    img = seq.augment_image(img)
                
                img = cv2.resize(img, (img_rows, img_cols))
                batch_data[j] = img
                batch_labels[j] = to_categorical(label, num_classes=nb_classes)

                if preprocessing_function is not None:
                    batch_data = preprocessing_function(batch_data)

                yield batch_data, batch_labels


In [ ]:
#VGG16 model

preprocessing_function = vgg16.preprocess_input

train_data_gen = data_generator(train_df, batch_size, preprocessing_function)
valid_data_gen = data_generator(valid_df, batch_size, preprocessing_function, is_validation_data=True)



In [ ]:
def get_base_model():
    base_model = vgg16.VGG16(
        input_shape=(img_rows, img_cols, img_channels), 
        weights='imagenet', 
        include_top=True #True means change the CNN, Flase means we will not change anything
        #Transfer learning with fine tuning
    )

    return base_model

In [ ]:
#Get the base model
base_model = get_base_model()

#Get output of second last dense layer
base_model_output = base_model.layers[-2].output

#Add new layers
x = L.Dropout(0.5, name='drop2')(base_model_output)
output = L.Dense(nb_classes, activation='softmax', name='fc3')(x)

#define a new model
model = Model(inputs = base_model.input, outputs = output)



In [ ]:
#Freeze all the base model layers

for layer in base_model.layers[:-1]:
    layer.trainable = False
    

In [ ]:
#Compile the model

optimizer = RMSprop(0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()



In [ ]:
#Model training

#Apply early stopping
es = EarlyStopping(patience=100, restore_best_weights=True)

#Checkpoint to save the model
chkpt = ModelCheckpoint(filepath='model_checkpoint.h5', save_best_only=True)

#No. of training and validation step for training and validation
nb_train_steps = int(np.ceil(len(train_df)/batch_size))
nb_valid_steps = int(np.ceil(len(valid_df)/batch_size))

# Nuber of epochs
nb_epochs = 50

#train model
hist = model.fit_generator(train_data_gen, epochs=nb_epochs, steps_per_epoch=nb_train_steps, 
                           validation_data=valid_data_gen, validation_steps=nb_valid_steps, 
                           callbacks=[es, chkpt])




In [ ]:
#EfficientNetB0 - To try

base_model = keras.applications.EfficientNetB0(include_top=True)

#Try - Xception model
